In [7]:
# Load Libraries for Python code:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from IPython.display import display, HTML
from io import BytesIO
import base64

# Upload your excel files to see your datasets:

**Instructions:**
1. **Upload Files:**
    - Click the "Choose Files" button.
    - Select one or multiple Excel files (**.xlsx**) from your computer.
    - Wait for the files to be uploaded. You will see the file names listed once they are uploaded.

2. **Select File:**
    - Use the dropdown menu labeled "Select File" to choose one of the uploaded Excel files.

3. **Process File:**
    - Click the "Process File" button.
    - The tool will read the selected Excel file and display its sheets and data frames.
    - You will see the sheet names followed by the data in each sheet.
    - To clear the "Process File", click the "Clear" button.

**Example Workflow:**
- Upload **file1.xlsx** and **file2.xlsx**.
- Select **file1.xlsx** from the "Select File" dropdown.
- Click the "Process File" button to see the contents of **file1.xlsx**.

In [9]:
# Widget to upload Excel files
upload = widgets.FileUpload(accept='.xlsx', multiple=True)

# Dropdown to select a file
file_dropdown = widgets.Dropdown(description='Select File')

# Button to process the selected file
process_button = widgets.Button(description='Process File')
clear_button = widgets.Button(description='Clear')

# Output area to display DataFrames
output = widgets.Output()

def read_excel(file_content):
    # Read the Excel file and return a dictionary of DataFrames by sheet name
    return pd.read_excel(BytesIO(file_content), sheet_name=None)

def display_dataframes(dataframes_dict, filename):
    for sheet_name, df in dataframes_dict.items():
        display(HTML(f"<h3>{filename} - {sheet_name}</h3>"))
        display(df)

def update_dropdown(change):
    # Update the dropdown menu with the names of the uploaded files
    if upload.value:
        file_dropdown.options = [(file['name'], file['name']) for file in upload.value]
    else:
        file_dropdown.options = []

def process_file(b):
    with output:
        output.clear_output()
        selected_file = file_dropdown.value
        if not selected_file:
            print("Please select a file from the dropdown.")
            return
        for file in upload.value:
            if file['name'] == selected_file:
                dataframes_dict = read_excel(file['content'])
                display_dataframes(dataframes_dict, file['name'])
                break

def clear_output_area(b):
    output.clear_output()

upload.observe(update_dropdown, names='value')
process_button.on_click(process_file)
clear_button.on_click(clear_output_area)

# Display the widgets
display(widgets.VBox([upload, file_dropdown, widgets.HBox([process_button, clear_button]), output]))


# ***
 -----------------------------------------------------------
 -----------------------------------------------------------
 -----------------------------------------------------------
 ---
 # ***

# Concatenative Data:

**Instructions:**
1. **Upload Files:**
    - Click the "Choose Files" button.
    - Select one or multiple Excel files (.xlsx) from your computer.
    - Wait for the files to be uploaded. You will see the file names listed once they are uploaded.

2. **Select Files:**
    - Use the "Select File(s)" box to choose one or multiple files. Hold Ctrl (or Cmd on Mac) to select multiple files.
    - If you select multiple files, sheets from all selected files will be available in the next step.

3. **Select Sheets:**
    - Use the "Select Sheet(s)" box to choose one or more sheets from the selected files. If you want to concatenate all sheets together, select the "All" option.

4. **Concatenate:**
    - Click the "Concatenate" button.
    - The tool will concatenate the data from the selected sheets and display the concatenated DataFrame.
    - The column names of the concatenated DataFrame will also be displayed.
    - To clear the "Concatenate", click the "Clear" button.

5. **Set Index Columns (Optional):**
    - Select columns from the "Set Index Columns" box to set as the hierarchical index.
    - Click the "Set Index" button to set the index and display the hierarchical DataFrame.
    - To clear the index selection, click the "Clear" button.

6. **Export to Excel:**
    - Enter a file name in the "New File Name" box.
    - Click the "Export to Excel" button to save the concatenated (or hierarchical) DataFrame as an Excel file with the given name.

**Example Workflow:**
- Upload file1.xlsx and file2.xlsx.
- Select both files from the "Select File(s)" box.
- Select specific sheets or "All" from the "Select Sheet(s)" box.
- Click "Concatenate" to see the concatenated DataFrame.
- Optionally set index columns and click "Set Index".
- Enter a file name and click "Export to Excel".

In [11]:
# Widget to upload Excel files
file_upload = widgets.FileUpload(accept='.xlsx', multiple=True)

# Output area to display DataFrames and concatenated DataFrame
output_area = widgets.Output()

def read_excel_content(file_content):
    # Read the Excel file and return a dictionary of DataFrames by sheet name
    return pd.read_excel(BytesIO(file_content), sheet_name=None)

def show_dataframes(dataframes_dict, filename):
    for sheet_name, df in dataframes_dict.items():
        display(HTML(f"<h3>{filename} - {sheet_name}</h3>"))
        display(df)

# Display the upload widget
display(widgets.VBox([file_upload, output_area]))

# Additional Widgets
file_multiselect = widgets.SelectMultiple(description='Select File(s)')
sheet_multiselect = widgets.SelectMultiple(description='Select Sheet(s)')
concat_files_button = widgets.Button(description='Concatenate')
clear_concat_button = widgets.Button(description='Clear')
export_file_button = widgets.Button(description='Export to Excel')
concat_output_area = widgets.Output()
columns_output_area = widgets.Output()
new_filename_input = widgets.Text(description='New File Name', placeholder='Enter file name')

# New widgets for setting index
index_columns_multiselect = widgets.SelectMultiple(description='Set Index Columns')
set_index_button = widgets.Button(description='Set Index')
clear_index_button = widgets.Button(description='Clear')
hierarchical_output_area = widgets.Output()
hierarchical_columns_output_area = widgets.Output()

def update_file_multiselect(change):
    if file_upload.value:
        file_multiselect.options = [(file['name'], file['name']) for file in file_upload.value]
    else:
        file_multiselect.options = []

def update_sheet_multiselect(change):
    selected_files = file_multiselect.value
    if selected_files:
        all_sheets = set()
        for file in file_upload.value:
            if file['name'] in selected_files:
                file_content = file['content']
                dataframes_dict = read_excel_content(file_content)
                all_sheets.update(dataframes_dict.keys())
        sheet_multiselect.options = [("All", "All")] + [(sheet_name, sheet_name) for sheet_name in all_sheets]
    else:
        sheet_multiselect.options = []

def concatenate_files(b):
    global concatenated_df
    with concat_output_area:
        concat_output_area.clear_output()
        columns_output_area.clear_output()
        hierarchical_output_area.clear_output()
        hierarchical_columns_output_area.clear_output()
        selected_files = file_multiselect.value
        if not selected_files or len(selected_files) < 1:
            print("Please select at least one file.")
            return
        selected_sheets = sheet_multiselect.value
        all_dataframes = []
        
        for file in file_upload.value:
            if file['name'] in selected_files:
                dataframes_dict = read_excel_content(file['content'])
                if "All" in selected_sheets:
                    all_dataframes.extend(dataframes_dict.values())
                else:
                    for sheet_name in selected_sheets:
                        if sheet_name in dataframes_dict:
                            all_dataframes.append(dataframes_dict[sheet_name])
        
        if all_dataframes:
            concatenated_df = pd.concat(all_dataframes, ignore_index=True)
            display(concatenated_df)
            
            # Display the column names of the concatenated dataframe
            with columns_output_area:
                display(HTML("<h3>Column Names:</h3>"))
                display(concatenated_df.columns)
            
            # Add Export Button, filename input, and index columns multiselect after displaying the concatenated dataframe
            index_columns_multiselect.options = [(col, col) for col in concatenated_df.columns]
            display(index_columns_multiselect)
            display(widgets.HBox([set_index_button, clear_index_button]))
            display(new_filename_input)
            display(export_file_button)
        else:
            print("No data to concatenate.")

def set_index_columns(b):
    global hierarchical_df
    with hierarchical_output_area:
        hierarchical_output_area.clear_output()
        hierarchical_columns_output_area.clear_output()
        selected_index_columns = index_columns_multiselect.value
        if not selected_index_columns:
            print("Please select at least one column to set as index.")
            return
        hierarchical_df = concatenated_df.set_index(list(selected_index_columns), drop=True)
        display(hierarchical_df)
        
        # Display the column names of the hierarchical dataframe
        with hierarchical_columns_output_area:
            display(HTML("<h3>Hierarchical DataFrame Column Names:</h3>"))
            display(hierarchical_df.columns)

def clear_index_selection(b):
    index_columns_multiselect.value = []
    hierarchical_output_area.clear_output()
    hierarchical_columns_output_area.clear_output()
    display(concatenated_df)

def clear_concat_output_area(b):
    concat_output_area.clear_output()
    columns_output_area.clear_output()
    hierarchical_output_area.clear_output()
    hierarchical_columns_output_area.clear_output()

def export_to_excel_file(b):
    selected_files = file_multiselect.value
    selected_sheets = sheet_multiselect.value
    all_dataframes = []
    
    for file in file_upload.value:
        if file['name'] in selected_files:
            dataframes_dict = read_excel_content(file['content'])
            if "All" in selected_sheets:
                all_dataframes.extend(dataframes_dict.values())
            else:
                for sheet_name in selected_sheets:
                    if sheet_name in dataframes_dict:
                        all_dataframes.append(dataframes_dict[sheet_name])
    
    if all_dataframes:
        concatenated_df = pd.concat(all_dataframes, ignore_index=True)
        selected_index_columns = index_columns_multiselect.value
        if selected_index_columns:
            hierarchical_df = concatenated_df.set_index(list(selected_index_columns), drop=True)
            df_to_export = hierarchical_df
        else:
            df_to_export = concatenated_df
        
        new_filename = new_filename_input.value if new_filename_input.value else 'concatenated_output'
        df_to_export.to_excel(f'{new_filename}.xlsx', index=False)
        with output_area:
            print(f"File exported as '{new_filename}.xlsx'.")

file_upload.observe(update_file_multiselect, names='value')
file_multiselect.observe(update_sheet_multiselect, names='value')
concat_files_button.on_click(concatenate_files)
clear_concat_button.on_click(clear_concat_output_area)
set_index_button.on_click(set_index_columns)
clear_index_button.on_click(clear_index_selection)
export_file_button.on_click(export_to_excel_file)

# Display the widgets for file and sheet selection, and concatenation
display(widgets.VBox([
    file_multiselect, 
    sheet_multiselect, 
    widgets.HBox([concat_files_button, clear_concat_button]), 
    concat_output_area, 
    columns_output_area, 
    hierarchical_output_area, 
    hierarchical_columns_output_area
]))


# ***
 -----------------------------------------------------------
 -----------------------------------------------------------
 -----------------------------------------------------------
 ---
 # ***

# Merging Data:

**Instructions:**
1. **Upload Files:**
    - Click the "Choose Files" button.
    - Select one or multiple Excel files (.xlsx) from your computer.
    - Wait for the files to be uploaded. You will see the file names listed once they are uploaded.

2. **Select First Dataset:**
    - Use the "Dataset 1" dropdown to select the first file.
    - Use the "Sheet 1" dropdown to select a sheet from the first file.

3. **Select Second Dataset:**
    - Use the "Dataset 2" dropdown to select the second file.
    - Use the "Sheet 2" dropdown to select a sheet from the second file.

4. **Match Columns:**
    - Enter column names to match on in the "Match on Columns" box, separated by commas.
    - If left empty, the merge will use all columns.

5. **Select Merge Method:**
    - Use the "Merge Method" dropdown to select the type of merge:
    - All (outer): Includes all data from both datasets.
    - Matched data (inner): Includes only matched data from both datasets.
    - Left-join with Matched: Includes all data from the first dataset and matched data from the second dataset.
    - Right-join with Matched: Includes all data from the second dataset and matched data from the first dataset.
    - Unmatched data: Includes only unmatched data from both datasets.

6. **Merge Datasets:**
    - Click the "Merge Datasets" button.
    - The tool will merge the selected sheets and display the merged DataFrame.
    - The column names of the merged DataFrame will also be displayed.
    - To clear the "Merge Datasets", click the "Clear" button.

7. **Set Index Columns (Optional):**
    - Select columns from the "Set Index Column(s)" box to set as the hierarchical index.
    - Click the "Proceed" button to set the index and display the hierarchical DataFrame.
    - To clear the index selection, click the "Clear" button.

8. **Export to Excel:**
    - Enter a file name in the "New File Name" box.
    - Click the "Export to Excel" button to save the merged (or hierarchical) DataFrame as an Excel file with the given name.

**Example Workflow:**
- Upload file1.xlsx and file2.xlsx.
- Select file1.xlsx in "Dataset 1" and choose a sheet in "Sheet 1".
- Select file2.xlsx in "Dataset 2" and choose a sheet in "Sheet 2".
- Enter columns to match on or leave empty to use all columns.
- Choose a merge method.
- Click "Merge Datasets" to see the merged DataFrame.
- Optionally set index columns and click "Proceed".
- Enter a file name and click "Export to Excel".

In [13]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
from io import BytesIO

# Global variable to store merged dataframe
merged_dataframe = None

# Widget to upload Excel files
merge_upload = widgets.FileUpload(accept='.xlsx', multiple=True)

# Dropdowns to select two datasets (sheets)
merge_dataset1_select = widgets.Dropdown(description='Dataset 1:')
merge_sheet1_select = widgets.Dropdown(description='Sheet 1:')
merge_dataset2_select = widgets.Dropdown(description='Dataset 2:')
merge_sheet2_select = widgets.Dropdown(description='Sheet 2:')

# Textbox to input columns to match on
merge_columns_text = widgets.Text(description='Match on Columns (comma-separated):', placeholder='Leave empty to merge all columns')

# Dropdown to select merging method
merge_method_select = widgets.Dropdown(
    options=[
        ('All (outer)', 'outer'),
        ('Matched data (inner)', 'inner'),
        ('Left-join with Matched', 'left'),
        ('Right-join with Matched', 'right'),
        ('Unmatched data', 'unmatched')
    ],
    description='Merge Method:'
)

# Button to perform merging
merge_action_button = widgets.Button(description='Merge Datasets')
clear_merge_button = widgets.Button(description='Clear')

# Output areas
merge_output = widgets.Output()
merged_dataframe_output = widgets.Output()
merge_columns_output = widgets.Output()
merge_filename_text = widgets.Text(description='New File Name', placeholder='Enter file name')
merge_export_action_button = widgets.Button(description='Export to Excel')

# Widgets for hierarchical index setting
merge_index_columns_select = widgets.SelectMultiple(description='Set Index Column(s)')
merge_proceed_button = widgets.Button(description='Proceed')
merge_clear_index_button = widgets.Button(description='Clear')
merge_hierarchical_output = widgets.Output()

def read_merge_excel(file_content):
    return pd.read_excel(BytesIO(file_content), sheet_name=None)

def update_merge_dropdowns(change):
    file_names = [(file['name'], file['name']) for file in merge_upload.value]
    merge_dataset1_select.options = file_names
    merge_dataset2_select.options = file_names

def update_merge_sheets(change):
    file_content = next(file['content'] for file in merge_upload.value if file['name'] == merge_dataset1_select.value)
    dataframes_dict = read_merge_excel(file_content)
    sheet_names = [(sheet_name, sheet_name) for sheet_name in dataframes_dict.keys()]
    merge_sheet1_select.options = sheet_names

    file_content = next(file['content'] for file in merge_upload.value if file['name'] == merge_dataset2_select.value)
    dataframes_dict = read_merge_excel(file_content)
    sheet_names = [(sheet_name, sheet_name) for sheet_name in dataframes_dict.keys()]
    merge_sheet2_select.options = sheet_names

def merge_datasets(b):
    global merged_dataframe
    with merged_dataframe_output:
        merged_dataframe_output.clear_output()
        merge_columns_output.clear_output()
        merge_output.clear_output()
        merge_hierarchical_output.clear_output()

        # Read the selected sheets
        file_content1 = next(file['content'] for file in merge_upload.value if file['name'] == merge_dataset1_select.value)
        df1 = read_merge_excel(file_content1)[merge_sheet1_select.value]
        
        file_content2 = next(file['content'] for file in merge_upload.value if file['name'] == merge_dataset2_select.value)
        df2 = read_merge_excel(file_content2)[merge_sheet2_select.value]

        # Get columns to match on
        columns = [col.strip() for col in merge_columns_text.value.split(',')] if merge_columns_text.value else None

        # Perform merge
        merge_method = merge_method_select.value
        if merge_method == 'unmatched':
            merged_dataframe = pd.concat([df1.merge(df2, how='outer', indicator=True).query('_merge != "both"')]).drop('_merge', axis=1)
        else:
            merged_dataframe = df1.merge(df2, how=merge_method, on=columns)

        # Display merged dataframe
        display(merged_dataframe)

        # Display column names
        with merge_columns_output:
            display(HTML("<h3>Column Names:</h3>"))
            display(merged_dataframe.columns)
            merge_index_columns_select.options = list(merged_dataframe.columns)
        
        # Add widgets for setting index columns, filename input, proceed button, clear button, and export button
        display(merge_index_columns_select)
        display(widgets.HBox([merge_proceed_button, merge_clear_index_button]))
        display(merge_filename_text)
        display(merge_export_action_button)

def set_merge_hierarchical_index(b):
    with merge_hierarchical_output:
        merge_hierarchical_output.clear_output()
        selected_index_columns = merge_index_columns_select.value
        if selected_index_columns:
            hierarchical_df = merged_dataframe.set_index(list(selected_index_columns), drop=True, append=False)
            display(hierarchical_df)
            display(HTML("<h3>Column Names after setting index:</h3>"))
            display(hierarchical_df.columns)
        else:
            print("No index columns selected. Displaying merged DataFrame.")
            display(merged_dataframe)

def clear_merge_index_selection(b):
    merge_index_columns_select.value = []
    merge_hierarchical_output.clear_output()
    display(merged_dataframe)

def clear_merge_output_area(b):
    merged_dataframe_output.clear_output()
    merge_columns_output.clear_output()
    merge_output.clear_output()
    merge_hierarchical_output.clear_output()

def export_merged_dataframe_to_excel(b):
    selected_index_columns = merge_index_columns_select.value
    if selected_index_columns:
        hierarchical_df = merged_dataframe.set_index(list(selected_index_columns), drop=True, append=False)
        new_filename = merge_filename_text.value if merge_filename_text.value else 'hierarchical_output'
        hierarchical_df.to_excel(f'{new_filename}.xlsx')
        with merge_output:
            print(f"File exported as '{new_filename}.xlsx'.")
    else:
        new_filename = merge_filename_text.value if merge_filename_text.value else 'merged_output'
        merged_dataframe.to_excel(f'{new_filename}.xlsx', index=False)
        with merge_output:
            print(f"File exported as '{new_filename}.xlsx'.")

merge_upload.observe(update_merge_dropdowns, names='value')
merge_dataset1_select.observe(update_merge_sheets, names='value')
merge_dataset2_select.observe(update_merge_sheets, names='value')
merge_action_button.on_click(merge_datasets)
clear_merge_button.on_click(clear_merge_output_area)
merge_proceed_button.on_click(set_merge_hierarchical_index)
merge_clear_index_button.on_click(clear_merge_index_selection)
merge_export_action_button.on_click(export_merged_dataframe_to_excel)

# Display the widgets
display(widgets.VBox([
    merge_upload,
    widgets.HBox([merge_dataset1_select, merge_sheet1_select]),
    widgets.HBox([merge_dataset2_select, merge_sheet2_select]),
    merge_columns_text,
    merge_method_select,
    widgets.HBox([merge_action_button, clear_merge_button]),
    merged_dataframe_output,
    merge_columns_output,
    merge_hierarchical_output,
    merge_output
]))


# ***
 -----------------------------------------------------------
 -----------------------------------------------------------
 -----------------------------------------------------------
 ---
 # ***

# References:

**geeksforgeeks** website:

- https://www.geeksforgeeks.org/python-pandas-merging-joining-and-concatenating/

Youtube videos to understand **Concatenate** and **Merging**:

- https://www.youtube.com/watch?v=wzN1UyfRSWI&t=390s

- https://www.youtube.com/watch?v=WGOEFok1szA&pp=ygUbbWVyZ2UgYW5kIGNvbmNhdCBpbiBwYW5kYXMg

- https://www.youtube.com/watch?v=h4hOPGo4UVU&t=274s&pp=ygUbbWVyZ2UgYW5kIGNvbmNhdCBpbiBwYW5kYXMg

In [17]:
!pip freeze > requirements.txt
